# Importing necessary libraries 

In [1]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from skimage.feature import hog
from PIL import Image
import numpy as np
import random
import pickle
import glob
import time
import cv2

Wall time: 34.4 s



# Step 1 : Reading Dataset images

In [2]:
%%time
print(' Reading vehicles images...')

vehicle =  glob.glob('./vehicles/*/*.png')
gti_right =  glob.glob('./GTI_Right/*.png')
vehicle_images = np.concatenate([vehicle, gti_right])

vehicle_images_original = []
for imagePath in vehicle_images:
    readImg = cv2.imread(imagePath)
    Img = cv2.cvtColor(readImg, cv2.COLOR_BGR2GRAY)
    Img = cv2.resize(Img,(32, 32))
    vehicle_images_original.append(Img)
    
print(' Done !')
#imgPlot = plt.imshow(vehicle_images_original[3400])
#print(vehicle_images_original[3400].shape)
vehicles_img = np.asarray(vehicle_images_original)
print(vehicles_img.shape)

 Reading vehicles images...
 Done !
(9456, 32, 32)
Wall time: 1min 9s


In [3]:
%%time
print(' Reading non-vehicles images...')

non_vehicle_images = glob.glob('./non-vehicles/Extras/*.png')

non_vehicle_images_original = []
for imagePath in non_vehicle_images:
    readImg = cv2.imread(imagePath)
    Img = cv2.cvtColor(readImg, cv2.COLOR_BGR2GRAY)
    Img = cv2.resize(Img,(32, 32))
    non_vehicle_images_original.append(Img)
    
print(' Done\n')

non_vehicles_img = np.asarray(non_vehicle_images_original)
print(non_vehicles_img.shape)

print("N° of vehicle images loaded : " + str(len(vehicle_images_original)))
print("N° of non-vehicle images loaded : " + str(len(non_vehicle_images_original)))

 Reading non-vehicles images...
 Done

(5068, 32, 32)
N° of vehicle images loaded : 9456
N° of non-vehicle images loaded : 5068
Wall time: 45.4 s


# Step 2 : Extract Features 

## 2-1 : HOG Features Extracting functions

In [4]:
%%time
# Return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        hog_features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, visualize=vis, feature_vector=feature_vec)
        return hog_features, hog_image
    # Otherwise call with one output
    else:      
        hog_features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                      transform_sqrt=True, visualize=vis, feature_vector=feature_vec)
        return hog_features

# Return extracted features 
def ExtractFeatures(images, orient, pix_per_cell, cell_per_block):
    featureList = []
    for Img in images:
        local_features = get_hog_features(Img, orient, pix_per_cell, cell_per_block, False, True)
        featureList.append(local_features)
    return featureList


Wall time: 0 ns


## 2-1 Extracting Features from Dataset 

In [11]:
%%time
orient = 16
pixels = 8
cells = 2
# Extract features for vehicle images 
vehicleFeatures= ExtractFeatures(vehicle_images_original, orient, pixels, cells)

Wall time: 13.1 s


In [12]:
%%time
# Extract features for non-vehicle images
nonVehicleFeatures = ExtractFeatures(non_vehicle_images_original, orient, pixels, cells)

Wall time: 4.19 s


In [13]:
%%time
featuresList= np.vstack([vehicleFeatures, nonVehicleFeatures])
print("Shape of features list is ", featuresList.shape)
labelList= np.concatenate([np.ones(len(vehicleFeatures)), np.zeros(len(nonVehicleFeatures))])
print("Shape of the label list is ", labelList.shape)

Shape of features list is  (14524, 576)
Shape of the label list is  (14524,)
Wall time: 310 ms


# Step 3 : Data Preprocessing

In [14]:
%%time
""" Data Preprocessing """
# train test split of data
X_train,  X_test1,Y_train, Y_test1 = train_test_split(featuresList, labelList, test_size=0.4)

X_valid, X_test, Y_valid, Y_test = train_test_split(X_test1, Y_test1, test_size=0.5)
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

(8714, 576)
(2905, 576)
(2905, 576)
Wall time: 723 ms


In [15]:
%%time
#normalization and scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled= scaler.transform(X_train)
X_test_scaled= scaler.transform(X_test)
X_valid_scaled= scaler.transform(X_valid)

print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(X_valid_scaled.shape)

(8714, 576)
(2905, 576)
(2905, 576)
Wall time: 394 ms


In [16]:
%%time
""" Define and Train a classifier """
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

classifier1= LinearSVC(dual = False)
classifier1.fit(X_train,Y_train)
print("training Accuracy is  ", classifier1.score(X_train,Y_train) )
#print("test Accuracy is  ", classifier1.score(X_test,Y_test) )
print("validation Accuracy is  ", classifier1.score(X_valid,Y_valid) )
Y_pred = classifier1.predict(X_test)
cm = confusion_matrix(Y_test, Y_pred)
print("Confusion matrix :\n",cm)

cr = classification_report(Y_test, Y_pred)
print("\nClassification report :\n",cr)

#cv_score = cross_val_score(classifier1,X_valid,Y_valid,cv = 10)
#print("validation Accuracy is ", cv_score.mean())

training Accuracy is   0.9613266008721597
validation Accuracy is   0.9490533562822719
Confusion matrix :
 [[ 914   98]
 [  68 1825]]

Classification report :
               precision    recall  f1-score   support

         0.0       0.93      0.90      0.92      1012
         1.0       0.95      0.96      0.96      1893

    accuracy                           0.94      2905
   macro avg       0.94      0.93      0.94      2905
weighted avg       0.94      0.94      0.94      2905

Wall time: 994 ms


In [17]:
%%time
filename = 'classifier2_Linearsvm_16_8_2.xml'
pickle.dump(classifier1, open(filename, 'wb'))

Wall time: 368 ms


In [140]:
%%time
from sklearn.model_selection import cross_val_score
# load the model from disk

classifier1 = pickle.load(open(filename, 'rb'))
scores = cross_val_score(classifier1,X_valid,Y_valid,cv=5 )
print("accuracy of validation is :", scores)


accuracy of validation is : [0.93248945 0.93389592 0.91549296 0.91267606 0.92112676]
Wall time: 14.8 s
